In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
start_date = "2019-03-01"
end_date = "2019-04-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  comments = pd.concat(comments)


/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  issues = pd.concat(issues)
/c/Users/chold/Dropbox/projects/jupyter-activity-snapshot/templates/mod/__init__.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prs = pd.concat(prs)


In [6]:
# Only keep the dates we want
comments = comments.query('updatedAt > @start_date and updatedAt < @end_date')
issues = issues.query('updatedAt > @start_date and updatedAt < @end_date')
prs = prs.query('updatedAt > @start_date and updatedAt < @end_date')

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/ipython](https://github.com/ipython)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)
* [github.com/nteract](https://github.com/nteract)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-03-01** to **2019-04-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 31 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                      repo  author
  46          nteract                  semiotic       4
  47          nteract             semiotic-docs       4
  45          nteract                 papermill       3
  48          nteract                      vdom       3
  43          nteract                  hydrogen       1
  44          nteract           meeting-minutes       1
  40       jupyterlab                jupyterlab      65
  41       jupyterlab            jupyterlab-git      10
  42       jupyterlab         jupyterlab-github       5
  32       jupyterhub                jupyterhub      24
  39       jupyterhub    zero-to-jupyterhub-k8s      24
  33       jupyterhub       mybinder.org-deploy      20
  36       jupyterhub              team-compass      12
  30       jupyterhub   configurable-http-proxy      10
  28       jupyterhub                    binder       8
  29       jupyterhub                 binderhub       8
  35       jupyterhub               nbgitpuller       8
  37       jupyterhub   the-littlest-jupyterhub       6
  34       jupyterhub       nativeauthenticator       4
  31       jupyterhub             dockerspawner       2
  38       jupyterhub               yarnspawner       2
  25  jupyter-widgets                ipyleaflet      32
  27  jupyter-widgets    widget-ts-cookiecutter       8
  26  jupyter-widgets       widget-cookiecutter       4
  14          jupyter              jupyter-book      23
  11          jupyter        enterprise_gateway      15
  22          jupyter               repo2docker      15
  21          jupyter                  notebook      14
  9           jupyter             docker-stacks      13
  10          jupyter     enhancement-proposals       7
  20          jupyter                    nbdime       5
  24          jupyter                 terminado       3
  13          jupyter                   jupyter       2
  15          jupyter         jupyter.github.io       2
  8           jupyter             accessibility       1
  12          jupyter                      help       1
  16          jupyter            jupyter_server       1
  17          jupyter            kernel_gateway       1
  18          jupyter                     nb2kg       1
  19          jupyter                 nbconvert       1
  23          jupyter  sphinxcontrib_github_alt       1
  6           ipython                   ipython      15
  7           ipython                 traitlets       3
  0   binder-examples               apt_install       6
  1   binder-examples                     conda       2
  2   binder-examples          continuous-build       2
  3   binder-examples                demo-julia       2
  4   binder-examples        minimal-dockerfile       2
  5   binder-examples              requirements       2,
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['semiotic', 'semiotic-docs', 'papermill', 'vdom', 'hydrogen', 'meeting-minutes', 'jupyterlab', 'jupyterlab-git', 'jupyterlab-github', 'jupyterhub', 'zero-to-jupyterhub-k8s', 'mybinder.org-deploy', 'team-compass', 'configurable-http-proxy', 'binder', 'binderhub', 'nbgitpuller', 'the-littlest-jupyterhub', 'nativeauthenticator', 'dockerspawner', 'yarnspawner', 'ipyleaflet', 'widget-ts-cookiecutter', 'widget-cookiecutter', 'jupyter-book', 'enterprise_gateway', 'repo2docker', 'notebook', 'docker-stacks', 'enhancement-proposals', 'nbdime', 'terminado', 'jupyter', 'jupyter.github.io', 'accessibility', 'help', 'jupyter_server', 'kernel_gateway', 'nb2kg', 'nbconvert', 'sphinxcontrib_github_alt', 'ipython', 'traitlets', 'apt_install', 'conda', 'continuous-build', 'demo-julia', 'minimal-dockerfile', 'requirements'],
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merged PRs in the last 31 days'
})

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:        username  authored authorAssociation
    4        kannes         6       CONTRIBUTOR
    0       betatim         4            MEMBER
    1  davidanthoff         2       CONTRIBUTOR
    2       davidar         2       CONTRIBUTOR
    3     emigre459         2       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:      username  authored authorAssociation
    0   fchapoton         6       CONTRIBUTOR
    1     jsnydes         3       CONTRIBUTOR
    2     lodagro         3       CONTRIBUTOR
    3  matangover         3       CONTRIBUTOR
    4   ndtallant         3       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:            username  authored       authorAssociation
    10        choldgraf        16                  MEMBER
    31         lresende        13                  MEMBER
    33            minrk        12                  MEMBER
    7           betatim         7                  MEMBER
    35          parente         7                  MEMBER
    25        javabrett         4             CONTRIBUTOR
    15     davidanthoff         3             CONTRIBUTOR
    24        jasmainak         3             CONTRIBUTOR
    26         jhermann         2             CONTRIBUTOR
    27       joergbrech         2             CONTRIBUTOR
    29      kevin-bates         2                  MEMBER
    16        ellisonbg         2                  MEMBER
    39         willingc         2                  MEMBER
    9      captainsafia         2                  MEMBER
    37          vidartf         2            COLLABORATOR
    4           Zsailer         2                  MEMBER
    2            Godley         2             CONTRIBUTOR
    32  maartenbreddels         1             CONTRIBUTOR
    36          romainx         1             CONTRIBUTOR
    30     lisamartin72         1             CONTRIBUTOR
    28            jtpio         1             CONTRIBUTOR
    38        vilhelmen         1             CONTRIBUTOR
    34          mwcraig         1             CONTRIBUTOR
    0            00Kai0         1                    NONE
    23      jakelevirne         1  FIRST_TIME_CONTRIBUTOR
    22           hugovk         1             CONTRIBUTOR
    21    gitter-badger         1             CONTRIBUTOR
    1       Athanasiamo         1             CONTRIBUTOR
    19        fchapoton         1             CONTRIBUTOR
    18         emiliotl         1             CONTRIBUTOR
    17          emdupre        

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org                repo   username  closed
    0  binder-examples         apt_install    betatim       4
    1  binder-examples         apt_install  yuvipanda       2
    2  binder-examples               conda  choldgraf       2
    3  binder-examples    continuous-build    betatim       2
    4  binder-examples          demo-julia    betatim       2
    5  binder-examples  minimal-dockerfile      minrk       2
    6  binder-examples        requirements    betatim       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:        org       repo username  closed
    7  ipython    ipython  Carreau      15
    8  ipython  traitlets    minrk       3,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:         org                      repo       username  closed
    9   jupyter             accessibility      ellisonbg       1
    10  jupyter             docker-stacks        parente      13
    11  jupyter     enhancement-proposals        Zsailer       1
    12  jupyter     enhancement-proposals   captainsafia       1
    13  jupyter     enhancement-proposals        parente       2
    14  jupyter     enhancement-proposals       willingc       3
    15  jupyter        enterprise_gateway    kevin-bates       6
    16  jupyter        enterprise_gateway       lresende       9
    17  jupyter                      help        Carreau       1
    18  jupyter                   jupyter      choldgraf       1
    19  jupyter                   jupyter       willingc       1
    20  jupyter              jupyter-book      choldgraf      23
    21  jupyter         jupyter.github.io        Carreau       1
    22  jupyter         jupyter.github.io           Ruv7       1
    23  jupyter            jupyter_server  SylvainCorlay       1
    24  jupyter            kernel_gateway       lresende       1
    25  jupyter                     nb2kg    kevin-bates       1
    26  jupyter                 nbconvert      takluyver       1
    27  jupyter                    nbdime        vidartf       5
    28  jupyter                  notebook          minrk      11
    29  jupyter                  notebook      takluyver       1
    30  jupyter                  notebook       willingc       2
    31  jupyter               repo2docker        betatim       8
    32  jupyter               repo2docker      choldgraf       2
    33  jupyter               repo2docker          minrk       3
    34  jupyter               repo2docker       willingc       1
    35  jupyter               repo2docker      yuvipanda       1
    36  jupyter  sphinxcontrib_github_alt      takluyver       1
    37  jupyter                 terminado      takluyver       3,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 31 days',
    width: 1000
  }), Chart({
    data:                 org                    repo     username  closed
    38  jupyter-widgets              ipyleaflet  martinRenou      32
    39  jupyter-widgets     widget-cookiecutter      vidartf       4
    40  jupyter-widgets  widget-ts-cookiecutter      vidartf       8,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
 

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                      repo  number  \
  0   binder-examples                demo-julia       2   
  1           ipython                 ipykernel       9   
  2           ipython               ipyparallel       6   
  3           ipython                   ipython      39   
  4           jupyter        enterprise_gateway       1   
  5           jupyter                      help      23   
  6           jupyter                   jupyter       4   
  7           jupyter              jupyter-book       5   
  8           jupyter         jupyter.github.io       1   
  9           jupyter            jupyter_client       3   
  10          jupyter           jupyter_console       2   
  11          jupyter       jupyter_kernel_test       1   
  12          jupyter                 nbconvert       1   
  13          jupyter                    nbdime       1   
  14          jupyter                  nbformat       2   
  15          jupyter                  notebook      37   
  16          jupyter                 qtconsole       1   
  17          jupyter               repo2docker       6   
  18  jupyter-widgets                ipyleaflet       8   
  19  jupyter-widgets                ipywidgets      36   
  20  jupyter-widgets       widget-cookiecutter       8   
  21       jupyterhub                    binder       2   
  22       jupyterhub                 binderhub       7   
  23       jupyterhub             dockerspawner       4   
  24       jupyterhub    jupyter-rsession-proxy       2   
  25       jupyterhub      jupyter-server-proxy       6   
  26       jupyterhub                jupyterhub      16   
  27       jupyterhub  jupyterhub-deploy-docker       2   
  28       jupyterhub               kubespawner      10   
  29       jupyterhub         ldapauthenticator       2   
  30       jupyterhub       mybinder.org-deploy       4   
  31       jupyterhub       nativeauthenticator       1   
  32       jupyterhub            oauthenticator       6   
  33       jupyterhub              team-compass       6   
  34       jupyterhub    zero-to-jupyterhub-k8s      17   
  35       jupyterlab         jupyter-renderers       5   
  36       jupyterlab                jupyterlab     113   
  37       jupyterlab            jupyterlab-git      30   
  38       jupyterlab          jupyterlab-latex      10   
  39       jupyterlab            jupyterlab_app       5   
  40          nteract              kernel-relay      10   
  41          nteract                   nteract       7   
  42          nteract                 papermill       2   
  43          nteract                 scrapbook       3   
  44          nteract                  semiotic      45   
  
                                 org/repo  
  0            binder-examples/demo-julia  
  1                     ipython/ipykernel  
  2                   ipython/ipyparallel  
  3                       ipython/ipython  
  4            jupyter/enterprise_gateway  
  5                          jupyter/help  
  6                       jupyter/jupyter  
  7                  jupyter/jupyter-book  
  8             jupyter/jupyter.github.io  
  9                jupyter/jupyter_client  
  10              jupyter/jupyter_console  
  11          jupyter/jupyter_kernel_test  
  12                    jupyter/nbconvert  
  13                       jupyter/nbdime  
  14                     jupyter/nbformat  
  15                     jupyter/notebook  
  16                    jupyter/qtconsole  
  17                  jupyter/repo2docker  
  18           jupyter-widgets/ipyleaflet  
  19           jupyter-widgets/ipywidgets  
  20  jupyter-widgets/widget-cookiecutter  
  21                    jupyterhub/binder  
  22                 jupyterhub/binderhub  
  23             jupyterhub/dockerspawner  
  24    jupyterhub/jupyter-rsession-proxy  
  25      jupyterhub/jupyter-server-proxy  
  26                jupyterhub/jupyterhub  
  27  jupyterhub/jupyterhub-deploy-do

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                     repo  number  \
  0   binder-examples              apt_install       6   
  1           ipython                  ipython      57   
  2           ipython          ipython-website       3   
  3           jupyter            docker-stacks      48   
  4           jupyter       enterprise_gateway       8   
  5           jupyter                     help       8   
  6           jupyter                  jupyter       2   
  7           jupyter             jupyter-book       7   
  8           jupyter        jupyter.github.io       1   
  9           jupyter           jupyter_client       2   
  10          jupyter           jupyter_server       1   
  11          jupyter                    nb2kg       2   
  12          jupyter                nbconvert       8   
  13          jupyter                   nbdime       5   
  14          jupyter                 nbformat       2   
  15          jupyter                 nbviewer       1   
  16          jupyter                 notebook      28   
  17          jupyter                qtconsole       1   
  18          jupyter              repo2docker      12   
  19          jupyter                terminado       1   
  20  jupyter-widgets               ipyleaflet      36   
  21  jupyter-widgets               ipywidgets      12   
  22  jupyter-widgets   widget-ts-cookiecutter       4   
  23       jupyterhub                   binder      12   
  24       jupyterhub                binderhub      10   
  25       jupyterhub            dockerspawner       2   
  26       jupyterhub     jupyter-server-proxy       6   
  27       jupyterhub               jupyterhub      42   
  28       jupyterhub        ldapauthenticator       2   
  29       jupyterhub      mybinder.org-deploy      10   
  30       jupyterhub              nbgitpuller       6   
  31       jupyterhub             team-compass      24   
  32       jupyterhub  the-littlest-jupyterhub       6   
  33       jupyterhub   zero-to-jupyterhub-k8s      20   
  34       jupyterlab        jupyter-renderers       5   
  35       jupyterlab               jupyterlab     125   
  36       jupyterlab           jupyterlab-git      30   
  37       jupyterlab         jupyterlab-latex       5   
  38          nteract                       dx       1   
  39          nteract                 examples       1   
  40          nteract                 hydrogen      14   
  41          nteract             kernel-relay       2   
  42          nteract          meeting-minutes       1   
  43          nteract               nteract.io       1   
  44          nteract                papermill      13   
  45          nteract                 semiotic       1   
  46          nteract            semiotic-docs       1   
  
                                    org/repo  
  0              binder-examples/apt_install  
  1                          ipython/ipython  
  2                  ipython/ipython-website  
  3                    jupyter/docker-stacks  
  4               jupyter/enterprise_gateway  
  5                             jupyter/help  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                jupyter/jupyter.github.io  
  9                   jupyter/jupyter_client  
  10                  jupyter/jupyter_server  
  11                           jupyter/nb2kg  
  12                       jupyter/nbconvert  
  13                          jupyter/nbdime  
  14                        jupyter/nbformat  
  15                        jupyter/nbviewer  
  16                        jupyter/notebook  
  17                       jupyter/qtconsole  
  18                     jupyter/repo2docker  
  19                       jupyter/terminado  
  20              jupyter-widgets/ipyleaflet  
  21              jupyter-widgets/ipywidgets  
  22  jupyter-widgets/widget-ts-cookiecutter  
  23                       jupyterhub/binder  
  24                    

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org               repo     kind  count
    35   jupyterlab  jupyter-renderers  created    5.0
    36   jupyterlab         jupyterlab  created  113.0
    37   jupyterlab     jupyterlab-git  created   30.0
    38   jupyterlab   jupyterlab-latex  created   10.0
    39   jupyterlab     jupyterlab_app  created    5.0
    96   jupyterlab  jupyter-renderers   closed    5.0
    97   jupyterlab         jupyterlab   closed  125.0
    98   jupyterlab     jupyterlab-git   closed   30.0
    99   jupyterlab   jupyterlab-latex   closed    5.0
    100  jupyterlab     jupyterlab_app   closed    0.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                  org                    repo     kind  count
    18   jupyter-widgets              ipyleaflet  created    8.0
    19   jupyter-widgets              ipywidgets  created   36.0
    20   jupyter-widgets     widget-cookiecutter  created    8.0
    52   jupyter-widgets  widget-ts-cookiecutter  created    0.0
    79   jupyter-widgets              ipyleaflet   closed   36.0
    80   jupyter-widgets              ipywidgets   closed   12.0
    81   jupyter-widgets     widget-cookiecutter   closed    0.0
    113  jupyter-widgets  widget-ts-cookiecutter   closed    4.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyter-widgets',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:          org             repo     kind  count
    1    ipython        ipykernel  created    9.0
    2    ipython      ipyparallel  created    6.0
    3    ipython          ipython  created   39.0
    46   ipython  ipython-website  created    0.0
    62   ipython        ipykernel   closed    0.0
    63   ipython      ipyparallel   closed    0.0
    64   ipython          ipython   closed   57.0
    107  ipython  ipython-website   closed    3.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for ipython',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                  org         repo     kind  count
    0    binder-examples   demo-julia  created    2.0
    45   binder-examples  apt_i

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/choldgraf/anaconda/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                     repo   time_open  \
  0   binder-examples              apt_install    0.050544   
  1           ipython                  ipython    6.707002   
  2           ipython          ipython-website    0.600301   
  3           jupyter            docker-stacks  225.980266   
  4           jupyter       enterprise_gateway    1.738611   
  5           jupyter                     help    0.395110   
  6           jupyter                  jupyter    3.518900   
  7           jupyter             jupyter-book    0.135498   
  8           jupyter        jupyter.github.io    0.077083   
  9           jupyter           jupyter_client  244.560689   
  10          jupyter           jupyter_server    0.833322   
  11          jupyter                    nb2kg    0.530353   
  12          jupyter                nbconvert   73.019317   
  13          jupyter                   nbdime    2.972917   
  14          jupyter                 nbformat  289.033472   
  15          jupyter                 nbviewer    6.889421   
  16          jupyter                 notebook    3.707616   
  17          jupyter                qtconsole    2.596262   
  18          jupyter              repo2docker   18.533420   
  19          jupyter                terminado   14.735301   
  20  jupyter-widgets               ipyleaflet   23.043368   
  21  jupyter-widgets               ipywidgets    1.889109   
  22  jupyter-widgets   widget-ts-cookiecutter    1.047234   
  23       jupyterhub                   binder    0.895885   
  24       jupyterhub                binderhub   12.662407   
  25       jupyterhub            dockerspawner  453.596435   
  26       jupyterhub     jupyter-server-proxy   33.565995   
  27       jupyterhub               jupyterhub    6.655440   
  28       jupyterhub        ldapauthenticator    0.523206   
  29       jupyterhub      mybinder.org-deploy    0.679387   
  30       jupyterhub              nbgitpuller    0.447176   
  31       jupyterhub             team-compass   38.602483   
  32       jupyterhub  the-littlest-jupyterhub   50.864780   
  33       jupyterhub   zero-to-jupyterhub-k8s    9.446024   
  34       jupyterlab        jupyter-renderers  391.798669   
  35       jupyterlab               jupyterlab    1.762928   
  36       jupyterlab           jupyterlab-git   56.930868   
  37       jupyterlab         jupyterlab-latex    0.035741   
  38          nteract                       dx   43.691389   
  39          nteract                 examples   23.276771   
  40          nteract                 hydrogen    2.038843   
  41          nteract             kernel-relay    0.993947   
  42          nteract          meeting-minutes    7.155718   
  43          nteract               nteract.io    0.812778   
  44          nteract                papermill   13.431806   
  45          nteract                 semiotic    4.484850   
  46          nteract            semiotic-docs    7.700521   
  
                                    org/repo  
  0              binder-examples/apt_install  
  1                          ipython/ipython  
  2                  ipython/ipython-website  
  3                    jupyter/docker-stacks  
  4               jupyter/enterprise_gateway  
  5                             jupyter/help  
  6                          jupyter/jupyter  
  7                     jupyter/jupyter-book  
  8                jupyter/jupyter.github.io  
  9                   jupyter/jupyter_client  
  10                  jupyter/jupyter_server  
  11                           jupyter/nb2kg  
  12                       jupyter/nbconvert  
  13                          jupyter/nbdime  
  14                        jupyter/nbformat  
  15                        jupyter/nbviewer  
  16                        jupyter/notebook  
  17                       jupyter/qtconsole  
  18                     jupyter/repo2docker  
  19                       jupyter/terminado  
  20              ju

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 31 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             author  count authorAssociation
    1        choldgraf     14            MEMBER
    3           kannes     14       CONTRIBUTOR
    0          betatim     12            MEMBER
    4  matthewfeickert      8              NONE
    6        yuvipanda      6            MEMBER
    2     davidanthoff      4       CONTRIBUTOR
    5            minrk      2            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:                 author  count authorAssociation
    3              Carreau    153            MEMBER
    54         martinRenou     57              NONE
    66            rmorshea     33      COLLABORATOR
    55          matangover     18       CONTRIBUTOR
    71           tacaswell     18       CONTRIBUTOR
    51     maartenbreddels     15              NONE
    59               minrk     15            MEMBER
    76             travigd     12              NONE
    62         rainwoodman     12              NONE
    58            mimakaev      9              NONE
    29              daharn      9       CONTRIBUTOR
    56      mathieulongtin      9              NONE
    16        Viktor-Demin      9              NONE
    69              stas00      9              NONE
    26          ccordoba12      6       CONTRIBUTOR
    34           eranegozy      6              NONE
    45             jsnydes      6       CONTRIBUTOR
    64           redhermes      6              NONE
    35         flutefreak7      6              NONE
    46                krey      6              NONE
    12      LucianaMarques      6       CONTRIBUTOR
    11           KelSolaar      6              NONE
    72           takluyver      6            MEMBER
    75              tomyun      6       CONTRIBUTOR
    47            letalvoj      6              NONE
    32       drscotthawley      6              NONE
    53               mapio      3       CONTRIBUTOR
    52          mangecoeur      3              NONE
    50             litesam      3              NONE
    49        lisamartin72      3              NONE
    48        liquidcarbon      3              NONE
    0      AasthaTimalsina      3              NONE
    57         meeseeksdev      3       CONTRIBUTOR
    60           ndtallant      3       CONTRIBUTOR
    61          pabloslash      3              NONE
    65           rmbrannan      3              NONE
    67           softarmpl      3              NONE
    68          sonofagunn      3              NONE
    70            stevengj      3              NONE
    73           techtonik      3       CONTRIBUTOR
    74          texadactyl      3              NONE
    77           wuqiaomin      3              NONE
    78               yc-zc      3              NONE
    79           yiran7324      3              NONE
    63             red8012      3              NONE
    40           hriday535      3              NONE
    44  jorisvandenbossche      3       CONTRIBUTOR
    10             JockeTF      3              NONE
    19               abaar      3              NONE
    18            YubinXie      3              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['M

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             author  n_first_responses authorAssociation
    1        choldgraf                  3            MEMBER
    2           kannes                  3       CONTRIBUTOR
    0          betatim                  1            MEMBER
    3  matthewfeickert                  1              NONE
    4            minrk                  1            MEMBER
    5        yuvipanda                  1            MEMBER,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:                     author  n_first_responses authorAssociation
    1                  Carreau                 26            MEMBER
    41               tacaswell                  4       CONTRIBUTOR
    37                rmorshea                  4      COLLABORATOR
    22             flutefreak7                  2              NONE
    44                 travigd                  2       CONTRIBUTOR
    39                  stas00                  2              NONE
    31              matangover                  2       CONTRIBUTOR
    33               ndtallant                  1       CONTRIBUTOR
    26              jasonzhouu                  1              NONE
    27                    krey                  1              NONE
    28                letalvoj                  1              NONE
    29            liquidcarbon                  1              NONE
    30              mangecoeur                  1              NONE
    32                   minrk                  1            MEMBER
    0   AbsurdMagpieScrutinies                  1              NONE
    34              pabloslash                  1              NONE
    25                gpotter2                  1       CONTRIBUTOR
    36               rmbrannan                  1              NONE
    38              sonofagunn                  1              NONE
    40                stevengj                  1              NONE
    42               techtonik                  1       CONTRIBUTOR
    43                  tomyun                  1       CONTRIBUTOR
    45               wuqiaomin                  1              NONE
    35                 red8012                  1              NONE
    23            flying-sheep                  1       CONTRIBUTOR
    24                   funic                  1              NONE
    21              dwhswenson                  1              NONE
    2          DouglasRaillard                  1              NONE
    3                   Evidlo                  1              NONE
    4           GunaDhannarapu                  1              NONE
    5                KelSolaar                  1              NONE
    6           LucianaMarques                  1       CONTRIBUTOR
    7            Pierre-Bartet                  1              NONE
    8            SylvainCorlay                  1            MEMBER
    9             Viktor-Demin                  1              NONE
    10             VorpalBlade                  1              NONE
    11                   abaar                  1              NONE
    12            alexgawrilow                  1              NONE
    13                alexpdp7                  1              NONE
    14      

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>